In [1]:
%cd /kaggle/working

import os

from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="../experiments/032_feature_channel"):
    cfg = compose(
        config_name="config.yaml", overrides=["debug=True"], return_hydra_config=True
    )

/kaggle/working


In [2]:
import gc
import glob
import json
import math
import os
import pickle
import random
import shutil
import sys
import time
from pathlib import Path

import hydra
import numpy as np
import pandas as pd
import polars as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import webdataset as wds
from google.cloud import storage
from hydra.core.hydra_config import HydraConfig
from omegaconf import DictConfig, OmegaConf
from pytorch_lightning import (
    LightningDataModule,
    LightningModule,
    Trainer,
    seed_everything,
)
from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
    RichModelSummary,
    RichProgressBar,
)
from pytorch_lightning.loggers import WandbLogger
from sklearn.metrics import r2_score
from timm.utils import ModelEmaV2
from torch import nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import get_cosine_schedule_with_warmup

import utils
import wandb
from utils.metric import score

In [3]:
from pathlib import Path

import torch
import webdataset as wds
from google.cloud import storage
from pytorch_lightning import (
    LightningDataModule,
    LightningModule,
    Trainer,
    seed_everything,
)
from pytorch_lightning.loggers import WandbLogger

runtime_choices = cfg.hydra.runtime.choices
exp_name = f"notebook/{runtime_choices.exp}"
print(f"exp_name: {exp_name}")
output_path = Path(cfg.dir.output_dir) / exp_name
print(f"ouput_path: {output_path}")
os.makedirs(output_path, exist_ok=True)

exp_name: notebook/base
ouput_path: /kaggle/working/output/notebook/base


In [4]:
original_predict_df = pl.read_parquet(
    "gs://kaggle-leap/kami/experiments/054_adan/003/predict.parquet"
)

In [5]:
original_label_df = pl.read_parquet(
    "gs://kaggle-leap/kami/experiments/054_adan/003/label.parquet"
)

In [10]:
original_label_df.shape

(1451520, 369)

In [6]:
test_df = pl.read_parquet(cfg.exp.test_path)

In [7]:
def water_constraints(predict_df: pl.DataFrame, test_df: pl.DataFrame):
    predict_water = predict_df[:, 60:240].to_numpy()
    state_water = test_df[:, 61:241].to_numpy()  # considering test_id, add 1
    predict_water = np.clip(predict_water, None, state_water / 1200.0)
    predict_df = predict_df.with_columns(
        [
            pl.Series(name=col, values=predict_water[i])
            for i, col in enumerate(predict_df.columns[61:241])
        ]
    )

In [8]:
predict_df = water_constraints(original_predict_df, test_df)

ValueError: operands could not be broadcast together with shapes (1451520,180) (625000,180) 

In [5]:
# weight (weight zero もあるのでかけておく)
ss_df = pl.read_csv(
    "input/leap-atmospheric-physics-ai-climsim/sample_submission.csv", n_rows=1
)
weight_array = ss_df.select([x for x in ss_df.columns if x != "sample_id"]).to_numpy()[
    0
]

In [12]:
labels = original_label_df[:, 1:].to_numpy()
preds = original_predict_df[:, 1:].to_numpy()

In [14]:
predict_df = pd.DataFrame(
    preds * weight_array, columns=[i for i in range(preds.shape[1])]
).reset_index()

In [15]:
label_df = pd.DataFrame(
    labels * weight_array, columns=[i for i in range(labels.shape[1])]
).reset_index()

In [16]:
del preds, labels
gc.collect()

1121

In [17]:
r2_scores = score(label_df, predict_df, "index", multioutput="raw_values")
r2_score_dict = {
    col: r2 for col, r2 in dict(zip(cfg.cols.col_names, r2_scores)).items()
}

r2_score_dict = {
    col: r2 if col not in cfg.exp.fill_target else 1.0
    for col, r2 in r2_score_dict.items()
}

r2_score = float(np.array([v for v in r2_score_dict.values()]).mean())
print(f"{r2_score=}")

r2_score=0.7335064214929286
